In [1]:
%run init.ipynb

C:\Users\Administrator\Anaconda3\lib\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.CDSSM.get_default_preprocessor(
    ngram_size = 3
)

In [7]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 94/94 [00:00<00:00, 177.12it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 98/98 [00:00<00:00, 191.72it/s]
Building Vocabulary from a datapack.: 100%|██████████████████████████████████████| 886/886 [00:00<00:00, 888181.97it/s]
Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 94/94 [00:00<00:00, 169.77it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseToke

In [10]:
preprocessor.context

{'embedding_input_dim': 317,
 'filter_unit': <mzcn.preprocessors.units.frequency_filter.FrequencyFilter at 0x227230ee6d8>,
 'ngram_process_unit': <mzcn.preprocessors.units.ngram_letter.NgramLetter at 0x227229f3358>,
 'ngram_vocab_size': 317,
 'ngram_vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x227361f1470>,
 'vocab_size': 317,
 'vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x22736467f98>}

In [11]:
triletter_callback = mz.dataloader.callbacks.Ngram(preprocessor, mode='sum')
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    callbacks=[triletter_callback]
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed,
    callbacks=[triletter_callback]
)

In [12]:
padding_callback = mz.models.CDSSM.get_default_padding_callback(
    with_ngram=True,
    fixed_ngram_length=preprocessor.context['ngram_vocab_size']
)

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback,
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback,
)

In [13]:
model = mz.models.CDSSM()

model.params['task'] = ranking_task
model.params['vocab_size'] = preprocessor.context['ngram_vocab_size']
model.params['filters'] = 64
model.params['kernel_size'] = 3
model.params['conv_activation_func'] = 'tanh'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 64
model.params['mlp_num_fan_out'] = 64
model.params['mlp_activation_func'] = 'tanh'
model.params['dropout_rate'] = 0.8

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

CDSSM(
  (net_left): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(317, 64, kernel_size=(3,), stride=(1,))
    (2): Tanh()
    (3): Dropout(p=0.8, inplace=False)
    (4): AdaptiveMaxPool1d(output_size=1)
    (5): Squeeze()
    (6): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
    )
  )
  (net_right): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(317, 64, kernel_size=(3,), stride=(1,))
    (2): Tanh()
    (3): Dropout(p=0.8, inplace=False)
    (4): AdaptiveMaxPool1d(output_size=1)
    (5): Squeeze()
    (6): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tan

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=15
)

trainer.run()

[Iter-1 Loss-0.991]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2778 - normalized_discounted_cumulative_gain@5(0.0): 0.2778 - mean_average_precision(0.0): 0.2778



[Iter-2 Loss-0.992]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2778 - normalized_discounted_cumulative_gain@5(0.0): 0.2778 - mean_average_precision(0.0): 0.2778



[Iter-3 Loss-0.993]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2778 - normalized_discounted_cumulative_gain@5(0.0): 0.2778 - mean_average_precision(0.0): 0.2778



[Iter-4 Loss-0.982]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2778 - normalized_discounted_cumulative_gain@5(0.0): 0.2778 - mean_average_precision(0.0): 0.2778



[Iter-5 Loss-1.002]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2778 - normalized_discounted_cumulative_gain@5(0.0): 0.2778 - mean_average_precision(0.0): 0.2778



...

[Iter-11 Loss-0.982]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2778 - normalized_discounted_cumulative_gain@5(0.0): 0.2778 - mean_average_precision(0.0): 0.2778



[Iter-12 Loss-0.984]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2778 - normalized_discounted_cumulative_gain@5(0.0): 0.2778 - mean_average_precision(0.0): 0.2778



[Iter-13 Loss-0.989]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2778 - normalized_discounted_cumulative_gain@5(0.0): 0.2778 - mean_average_precision(0.0): 0.2778



[Iter-14 Loss-0.974]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2778 - normalized_discounted_cumulative_gain@5(0.0): 0.2778 - mean_average_precision(0.0): 0.2778



[Iter-15 Loss-0.976]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2778 - normalized_discounted_cumulative_gain@5(0.0): 0.2778 - mean_average_precision(0.0): 0.2778

Cost time: 5.310956716537476s


In [17]:
import gc
gc.collect()

1420